In [11]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

In [12]:
md_files = ["../data/mtg_rules.txt"]

In [24]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
    ("####", "h4"),
    ("#####", "h5"),
    ("######", "h6"),
    ("#######", "h7"),
]
paragraphs = []
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

all_splits = []
for file in md_files:
    if any([name in file for name in ["Changelog.md", "README.md", "Legal.md"]]):
        continue
    splits = file.split("/")
    with open(file, 'r') as f:
        md_header_splits = markdown_splitter.split_text(f.read())
        for text in md_header_splits:
            splits[-1] = splits[-1].replace(".md", "")
            txt_metadata = [f"{k}: {v}" for k, v in text.metadata.items()]
            text.metadata["File"] = splits[2]
            
            txt_metadata = [
                f"File: {splits[2]}",
                *txt_metadata,
            ]
            text.page_content = f"{'\n'.join(txt_metadata)}\n\n{text.page_content}"
        all_splits.extend(md_header_splits)

In [25]:
all_splits[1700]

Document(metadata={'h1': 'Magic: The Gathering Comprehensive Rules', 'h2': 'Glossary', 'h3': 'Space Sculptor', 'File': 'mtg_rules.txt'}, page_content='File: mtg_rules.txt\nh1: Magic: The Gathering Comprehensive Rules\nh2: Glossary\nh3: Space Sculptor\n\nA keyword ability that splits creatures on the battlefield into sectors. See rule 702.158, “Space Sculptor.”')

In [26]:
TOKEN = "OsSrZ1DqQh2Kdfamz0HP7CC7g1DYDZcEDRy_Ql_trCP81h5TnBov2oZ5cOAKSEemTiK9hQ8WdRhiQuPc"
PRODUCT_ID = "101420"
URL = f"https://api.infomaniak.com/1/ai/{PRODUCT_ID}/openai/v1"

In [27]:
from common.infomaniak.ik_embeddings import IKEmbeddings

model = "bge_multilingual_gemma2" # "mini_lm_l12_v2" # 

embeder = IKEmbeddings(model=model)
index = f"mtg_rules_{model}"

In [28]:
import os
from langchain.vectorstores import Meilisearch
import meilisearch

# You can use the same code as `setup.py` to check for missing env vars

# Create the vector store
client = meilisearch.Client(
    url=os.environ.get("MEILI_HTTP_ADDR", "http://localhost:7700"),
    api_key=os.environ.get("MEILI_API_KEY", "2AB8F223884CE"),
)

In [29]:
# client.delete_index("dnd_rules")
client.delete_index(index)

TaskInfo(task_uid=43, index_uid='mtg_rules_bge_multilingual_gemma2', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2025, 2, 17, 19, 39, 53, 716094))

In [30]:
from requests import patch

patch("http://localhost:7700/experimental-features", headers={
    "Authorization": f"Bearer {os.environ.get("MEILI_API_KEY", "2AB8F223884CE")}"
}, json={"vectorStore": True}).text

'{"vectorStore":true,"metrics":false,"logsRoute":false,"editDocumentsByFunction":false,"containsFilter":false}'

In [31]:
from requests import get

get("http://localhost:7700/version", headers={
    "Authorization": f"Bearer {os.environ.get("MEILI_API_KEY", "2AB8F223884CE")}"
}).text

'{"commitSha":"876084d48004e04ac0720cb064915d53c4743a1a","commitDate":"2025-01-30T10:52:23.000000000Z","pkgVersion":"1.12.8"}'

In [32]:
from tqdm import tqdm


embedders = { 
    "custom": {
        "source": "userProvided",
        "dimensions": 3584
    }
}
embedder_name = "custom" 
vector_store = Meilisearch(client=client, embedding=embeder, index_name=index, embedders=embedders)

batch_size = 300
for i in tqdm(range(0, len(all_splits), batch_size)):
    # vector_store = vector_store.add_documents(documents=all_splits, embedding=embeder, embedder_name=embedder_name)
    vector_store.add_documents(all_splits[i:i+batch_size], embedding=embeder, embedder_name=embedder_name)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:47<00:00,  6.81s/it]


In [33]:
index

'mtg_rules_bge_multilingual_gemma2'